# Main notebook

For quick and generic simulations.

## Imports

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# %matplotlib widget

import lppydsmc as ld
import plotting

# imports for poisson solver
from fenics import *
import lppydsmc.poisson_solver as ps


# other imports
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import os
from pathlib import Path

# you can choose the seed here
np.random.seed(1111)

## System choice

Four default systems can be initialized using the following cell.

In [4]:
# ---------------------- System --------------------
system_type = 'thruster_three_grids'
dz = 0.001
typical_lenght = 0.001 # typical size of the system (minimum distance between two walls for example)
                       # used for computing the mean free path later on (not used in the simulation)
                       # just useful to have an idea of it.

dp = 0.001
# here, x : l, y : w
if(system_type == 'thruster'):
    
    dict_thruster = {
        'w_in' : 5*dp,
        'l_in' : 3*dp,
        'w_1' : 3*dp,
        'l_1' : dp,
        'l_int' : dp,
        'w_2' : dp,
        'l_2' : 10*dp,
        'w_out' : 5*dp,
        'l_out' : dp,
        'offsets' : np.array([0,0]) 
    }

    system, idx_out_walls, idx_in_wall = ld.systems.helper.thruster_system(**dict_thruster)
    idx_out_walls = [10, 9, 11] # not the input wall

elif(system_type == 'thruster_three_grids'):
    dict_thruster = {
        'w_in' : 5*dp,
        'l_in' : 3*dp,
        'w_1' : 3*dp,
        'l_1' : dp,
        'l_int' : dp,
        'w_2' : dp,
        'l_2' : 10*dp,
        'l_int_2' : dp,
        'w_3' : 3*dp,
        'l_3' : 1*dp,
        'w_out' : 5*dp,
        'l_out' : dp,
        'offsets' : np.array([0,0]) 
    }
    system, idx_out_walls, idx_in_wall = ld.systems.helper.thruster_three_grids_system(**dict_thruster)
    idx_out_walls = [13, 14, 15] # not the input wall

offsets = system.get_offsets()
system_shape = system.system_shape()
a = system.get_dir_vects()
segments = system.get_segments()

In [5]:
# electric field and stuff
charge_density = {
            'value' : '0',  # must be a string too
            'degree' : 0,
            'kwargs' : {}
            }

potential_field, electric_field = ps.helper.thruster(dimensions = dict_thruster, mesh_resolution = 100, \
                                                     potential_electrode_1 = '30', potential_electrode_2 = '300', \
                                                     potential_electrode_3 = '-100', charge_density = charge_density)


## Particles

In [6]:
# ------------------ Particles Params ----------------- #

# particles density in the real system
density = 1e17 # m-3

part_type = 'I-'
charge, mass, radius = 1*ld.utils.physics.ELECTRON_CHARGE, ld.utils.physics.get_mass_part(53+1, 53, 74), 2e-10

temperature = 24e3 # K - ion speed is much higher (around 2000 m/s)
v_mean = ld.utils.physics.maxwellian_mean_speed(temperature, mass) # for 24k K , ~ 1993 m/s

size_array = 10000 # for example - to start with
container = ld.data_structures.Particle(part_type, charge, mass, radius, size_array)

# particle cross section (useful for particles collision)
cross_section = container.get_params()[3]  

# mean free path and time
mfp = ld.utils.physics.mean_free_path(cross_section, density)
mft = ld.utils.physics.mean_free_time(mfp, v_mean = v_mean) # min(typical_lenght,mfp)

In [7]:
ld.utils.physics.mean_free_time(typical_lenght, v_mean = 10*v_mean) # this is the actual time we need

5.0171189082561205e-08

## Injection params 

In [8]:
inject_particles = True

if(inject_particles):
    in_wall = segments[idx_in_wall]
    in_vect = np.array([a[idx_in_wall,1], -a[idx_in_wall,0]])
    
    # for the injection
    vel_std = ld.utils.physics.gaussian(temperature, mass)


## Reactions 

### with walls

function used : ld.advection.reactions.basic(arr, count, law) - not vectorized.
with :
* arr : the array of particle - shape : $N \times 5$ 
* count : the array of int - shape : $N$ - each value corresponds to the number of times the associated particle in *arr* collided with a wall.
* law : a function, params : (part = [x,y,vx,vy,vz], count), returns a probability.

The function returns an **array of reacting particles**.

*TODO* : in the future, we may want to take the angle with the wall.

Once we have the array, we have to change the type of the particles. We will use two container for now.

In [9]:
max_speed = 40000 # for example (we just normalize it by a "maximum value"- in theory it should corresponds to some realistic value)

def law_walls(part, c):
    proba = (np.linalg.norm(part[2:5])/max_speed)**c
    # print('Mean proba reacting with walls : {:.3e}'.format(np.mean(proba)))
    return proba

def law_walls_angle(part, c, angle):
    # proba = np.cos(angle)*(np.linalg.norm(part[2:5])/max_speed)**c
    # print('Mean proba reacting with walls : {:.3e}'.format(np.mean(proba)))
    return 1 # proba - reaction proba is now 1

In [10]:
# adding energy depletion - controlled by alpha
# simplest way for now
# in the future, we could take into account the angle the particle had with the wall
# however it depends on weather we consider the collision to be specular or diffusive
energy_depletion = True
alpha = 0.1 # to start with 

### with particles (or "background gas")

In this case, we consider that there is a certain probability of colliding and reacting depending on the position in the system and the speed of the particle. 

We are simply going to use : background_gas(arr, law)

Where :
- arr is the array of particles
- law the law that gives the probility of reactions (vectorized here)

#### Note : for now this is highly poorly coded, not usable in the long run etc. but it is still good just to try out and answer the problem we are facing.

In [11]:
max_value = 3.2e19 # m-3
min_value = 0 # m-3
min_x = 0.0
max_x = 1.8e-2
density_fn = lambda arr :  max_value - arr[:,0]/(max_x-min_x) * (max_value-min_value) # vectorized
density_fn_ = lambda X :  max_value - X/(max_x-min_x) * (max_value-min_value) 
def reacting_particles(arr):
    proba =  cross_section * density_fn(arr) * np.linalg.norm(arr[:,2:], axis = 1) * dt # if it collides then it necessarily reacts
    # print('Mean proba reacting with background gas : {:.3e}'.format(np.mean(proba)))
    return proba

In [12]:
%matplotlib widget
from plotting import analysis
fig, ax = plt.subplots(constrained_layout = True)
X = np.linspace(min_x, max_x, 1000)
ax.plot(X, density_fn_(X))
analysis.set_axis(ax, x = 'x', y = 'density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Transfering from one type to another
=> I will not actually use it.

I will simply store the type of each particle in another array (which is not the way it was intended to be use). 
Indeed, I met an issue : I can not transfer particles from one type to another and at the same time delete them without adding a "handler" of the indexes that are going to change. Thus this is kind of terrible.

One possibility would be to do both at the same time I guess.

In [11]:
def transfer(src, dest, idxes): # useless for now
    for i in idxes :
        dest.add(src.pop(i))
    # where both src and dest are containers

In [12]:
species_container = ld.data_structures.Container(size_array, number_of_elements = 0, dtype = int) # 0 if of specie 0, 1 if of specie 1 etc.

## Simulation parameters

In [13]:
# Simulation params
iterations = int(5e3)
dt = 1e-8 # in sec.

# saving params
saving_period = 100 # when do we save various data (see the simulation algo)
adding_period = 1 # when to we add to the dataframe that contains the particles position and velocity

# advection function - returns a 2D arrays containing the acceleration for each of the given particle
    # here arr is a N x 5 array. N particles, and for each one : x, y, vx, vy, vz is stored.
    
def f(arr, t, m, q, electric_field, species):
    der = np.zeros((arr.shape[0], 5)) # (vx, vy, ax, ay, az)
    fact  = q/m
    
    species_arr = species.get_array()
    # print('Mean specie : {}'.format(np.mean(species_arr)))
    for k, part in enumerate(arr):
        if(species_arr[k] == 0):
            try:
                der[k,2:4] =  fact * electric_field(part[:2]) # no acceleration on z
            except Exception as e:
                pass
                # der[k,2:4] = 0. # this is not even useful as we initialize at 0 the array der
        # else :
        #    der[k,2:4] = 0. # in this case we have an neutral - no acceleration for it.
    der[:,:2] = arr[:,2:4]
    return der

# args is given to euler_explicit and then given to *f* (the advection function) in addition to arr and dt.
# in our case, it is not needed. However, we could imagine a system with an electric field computed at the setup phase, 
# and we would like to give it as an args.
args = [mass, charge, electric_field, species_container]
scheme = ld.utils.schemes.rk4

## Summing-up and plotting

In [14]:
print(f'Initializing a system of type {system_type}, of shape {system_shape}.')
print(f'The number of particles (type {part_type}) in the system is {container.get_current()}.')
print(f'Mean free path : {mfp} m ; Mean free time : {mft} m')
print(f'The simulation lasts {iterations} iterations, with a time step of {dt} s. Simulation duration : {dt*iterations} s')
# Note:  HDF5 uses a different format than csv, and the size on the disk is much different that what is expected. Check out : https://support.hdfgroup.org/HDF5/doc/H5.intro.html
# Here, you can at least multiply by 4 the size (considering we save much more than )
print(f'Disk space usage for saving this simulation (counting ONLY the particles positions and speed) and considering that we save {size_array} particles each time is {iterations//adding_period*size_array*(5*4)/1024**2} MB.') 

Initializing a system of type thruster_three_grids, of shape [0.018 0.005].
The number of particles (type I-) in the system is 0.
Mean free path : 14.067442439954778 m ; Mean free time : 0.007057803145630173 m
The simulation lasts 5000 iterations, with a time step of 1e-08 s. Simulation duration : 5e-05 s
Disk space usage for saving this simulation (counting ONLY the particles positions and speed) and considering that we save 10000 particles each time is 953.67431640625 MB.


# Saving directory and name

In [15]:
# which directory is used to save the data
# and under what name.
dir_path = Path('results/')
name = 'neutralization_3_minus100.h5' 

saver = ld.data.saver.Saver(dir_path, name)

# Simulation

The next cell takes care of the simulation. It algo gives you an idea of the evolution of the number of particles in the system and of its very general state.

At the end, the *saver* which saves the data is closed and you can then analyse your simulation using *analysis.ipynb*.

In [16]:
df = pd.DataFrame(columns = ['x','y','vx','vy','vz','type']) # bucket for the particles - index of particles is the iteration number
tracking_out = pd.Series(dtype = int)
tracking_collisions = pd.Series(dtype = int)
mean_proba_walls = pd.Series(dtype = float)
mean_proba_gas = pd.Series(dtype = float)
df_out_particles = pd.DataFrame(columns = ['x','y','vx','vy','vz','type'])
df_collision_with_walls = pd.DataFrame(columns = ['x','y','type'])
df_collision_background_gas = pd.DataFrame(columns = ['x','y','type'])

# ------------------------- INJECTING PARTICLES ------------------------- 
remains = 0
debit = size_array/dt
new, remains = ld.injection.maxwellian(in_wall, in_vect, debit, vel_std, dt, remains) # injecting directly 1000 particles
container.add_multiple(new)
species_container.add_multiple(np.zeros(new.shape[0], dtype = int))

# SIMULATING
print('|{:^10}|{:^10}|{:^10}|'.format(' it ', ' INIT ', 'DEL '))
print('{:-^34}'.format(''))

t = 0.
for it in range(1,iterations+1): # tqdm
    n1 = container.get_current()
    # ---------------------------- PHASE : ADVECTING --------------------
        # MOVING PARTICLES
    arr = container.get_array()
    
    # neutrals = arr[np.where(species_container.get_array()==0)]
    # nrj_neutral_before = np.sum(np.linalg.norm(neutrals, axis = 1)**2)
    
    ld.advection.advect(arr, f, dt, t, args, scheme) # advect is inplace - it not the same f for both I and I-...
    
    # neutrals = arr[np.where(species_container.get_array()==0)]
    # nrj_neutral_after = np.sum(np.linalg.norm(neutrals, axis = 1)**2)
    # print('{}'.format(nrj_neutral_after/nrj_neutral_before))
    # adding the reacting with background gas here
    idxes_reacting_gas, mp_gas = ld.collision.reactions.background_gas(arr, reacting_particles)
    
    if(it%adding_period == 0 or it == iterations-1):
        # saving the position of the collision (and type)
        reacting_species_gas = species_container.get_array()[idxes_reacting_gas]
        reacting_arr_gas = arr[idxes_reacting_gas] # it's a copy
        df_collision_background_gas = df_collision_background_gas.append(pd.DataFrame(data=np.concatenate((reacting_arr_gas[:,:2], np.expand_dims(reacting_species_gas, axis = 1)), axis = 1), index=[it]*reacting_arr_gas.shape[0], columns = ['x','y','type']))

    # updating
    species_container.update(idxes_reacting_gas, 1)
    
        # HANDLING BOUNDARIES
    count_collisions_per_part = np.full(fill_value = 0, shape = arr.shape[0])
    count = np.full(fill_value = True, shape = arr.shape[0]) # True if the particle is outside the system (and thus collided)
    angles = np.full(fill_value = 0, shape = arr.shape[0], dtype = float)
    # all of them are supposed to be outside the system
    idxes_out = []
    c = 0
    collisions_with_walls = 0
    while(np.count_nonzero(count) > 0):  # np.sum(count, where = count == True) - does not work in python 3.7
        c+=1
        ct, cp, cos_alpha = ld.advection.wall_collision.handler_wall_collision_point(arr[count], segments, a) # handler_wall_collision(arr[count], segments, a, radius)
        count, idxes_out_, cos_alpha = ld.advection.wall_collision.make_collisions_out_walls(arr, a, ct, cp, idx_out_walls, count, cos_alpha = cos_alpha) # idxes_out : indexes of the particles (in arr) that got out of the system
        
        if(energy_depletion):
            arr[count, 2:] = np.sqrt(1-alpha)*arr[count, 2:] # only speed is depleted
            
        idxes_out.append(idxes_out_)

        count_collisions_per_part += count.astype(int) # 1-
        # the first one that is received is the number of particles colliding with walls.
        if(c == 1):
            collisions_with_walls = np.count_nonzero(count)
            # angles[count] = np.arccos(np.abs(cos_alpha)) # does not work cuz angles[count] is a copy of angles
            np.place(angles, count, np.arccos(np.abs(cos_alpha)))
            
    idxes_out = np.sort(np.concatenate(idxes_out))
    
    idxes_reacting_walls, mp_walls = ld.advection.reactions.angle_dependance(arr, count_collisions_per_part, angles, law = law_walls_angle)  
        
    if(it%adding_period == 0 or it == iterations-1):
        # saving
        reacting_species_walls = species_container.get_array()[idxes_reacting_walls]
        reacting_angles_walls = angles[idxes_reacting_walls]
        reacting_arr_walls = arr[idxes_reacting_walls] # it's a copy
        df_collision_with_walls = df_collision_with_walls.append(pd.DataFrame(data=np.concatenate((reacting_arr_walls[:,:2], np.expand_dims(reacting_species_walls, axis = 1), np.expand_dims(reacting_angles_walls, axis = 1)), axis = 1), index=[it]*reacting_arr_walls.shape[0], columns = ['x','y','type','angle']))
    
    # the reacting particles needs their "type" to be updated from 0 to 1.
    species_container.update(idxes_reacting_walls, 1)

    # deleting particles that are outside
    out_species = species_container.pop_multiple(idxes_out)
    out_arr = container.pop_multiple(idxes_out)
    df_out_particles = df_out_particles.append(pd.DataFrame(data=np.concatenate((out_arr, np.expand_dims(out_species, axis = 1)), axis = 1), index=[it]*out_arr.shape[0], columns = ['x','y','vx','vy','vz','type']))
    t += dt
    
    # ----------------------------- PLOTTING AND SAVING (OPTIONAL) ----------------------------- 
    if(it%adding_period == 0 or it == iterations-1):
        df = df.append(pd.DataFrame(data=np.concatenate((container.get_array(), np.expand_dims(species_container.get_array(), axis = 1)), axis = 1), index=[it]*container.get_array().shape[0], columns = ['x','y','vx','vy','vz','type']))
        tracking_out.loc[it] = len(idxes_out)
        tracking_collisions.loc[it] = int(collisions_with_walls)
        mean_proba_walls.loc[it] = float(mp_walls)
        mean_proba_gas.loc[it] = float(mp_gas)
        
    if(it%saving_period == 0 or it == iterations-1): # saving if last iteration too
        saver.save(it = it, append = {
                        'df_out_particles' : df_out_particles,
                        'df_collision_with_walls' : df_collision_with_walls,
                        'df_collision_background_gas' : df_collision_background_gas,
                        'df' : df,
                        'total_deleted' : tracking_out, # evolution of the number of deleted particles per cell (int)
                        'collisions_with_walls' : tracking_collisions, # number of collisions with walls - evolution
                        'mean_proba_walls' : mean_proba_walls,
                        'mean_proba_gas' : mean_proba_gas,
                    })
        # resetting dataframe to not use too much memory
        df = pd.DataFrame(columns = ['x','y','vx','vy','vz','type']) # bucket for the particles - index of particles is the iteration number
        tracking_out = pd.Series(dtype = int)
        tracking_collisions = pd.Series(dtype = int)
        mean_proba_walls = pd.Series(dtype = float)
        mean_proba_gas = pd.Series(dtype = float)
        df_out_particles = pd.DataFrame(columns = ['x','y','vx','vy','vz','type'])
        df_collision_with_walls = pd.DataFrame(columns = ['x','y','type'])
        df_collision_background_gas = pd.DataFrame(columns = ['x','y','type'])
        print('|{:^10}|{:^10}|{:^10}|'.format(it, n1, idxes_out.shape[0]))
    
    if(container.get_current()==0):
        print('All particles got out.')
        break
        
saver.close()

|    it    |   INIT   |   DEL    |
----------------------------------
|   100    |  10000   |    0     |
|   200    |   4171   |    1     |
|   300    |   4023   |    1     |
|   400    |   3871   |    7     |
|   500    |   3338   |    1     |
|   600    |   2995   |    8     |
|   700    |   2695   |    0     |
|   800    |   2438   |    3     |
|   900    |   2218   |    2     |
|   1000   |   2009   |    0     |
|   1100   |   1884   |    0     |
|   1200   |   1786   |    1     |
|   1300   |   1660   |    0     |
|   1400   |   1598   |    0     |
|   1500   |   1511   |    1     |
|   1600   |   1437   |    0     |
|   1700   |   1365   |    1     |
|   1800   |   1282   |    0     |
|   1900   |   1231   |    2     |
|   2000   |   1194   |    0     |
|   2100   |   1137   |    0     |
|   2200   |   1094   |    1     |
|   2300   |   1052   |    0     |
|   2400   |   1027   |    0     |
|   2500   |   998    |    1     |
|   2600   |   976    |    1     |
|   2700   |   950  